In [7]:
print('PySpark Version :'+spark.version)

big_list = range(10000)
rdd = sc.parallelize(big_list, 2)
odds = rdd.filter(lambda x: x % 2 != 0)
odds.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PySpark Version :2.4.8-amzn-2
[1, 3, 5, 7, 9]

In [8]:
sc.install_pypi_package("requests")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [9]:
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install ipython

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
import requests
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import col

# Send a GET request to the Aviationstack API to retrieve flight data
response = requests.get("http://api.aviationstack.com/v1/flights",
                        params={"access_key": "5cf29606576465e5ccb8081fb532387b", "dep_iata": "WAW"})
data = response.json()

# Define the schema for the DataFrame
schema = StructType([
    StructField("flight_date", StringType(), True),
    StructField("flight_status", StringType(), True),
    StructField("departure", StructType([
        StructField("airport", StructType([
            StructField("name", StringType(), True),
            StructField("iata", StringType(), True),
            StructField("icao", StringType(), True),
            StructField("latitude", DoubleType(), True),
            StructField("longitude", DoubleType(), True),
            StructField("geo_region", StringType(), True)
        ]), True),
        StructField("scheduled_time", StringType(), True),
        StructField("estimated_time", StringType(), True)
    ]), True),
    StructField("arrival", StructType([
        StructField("airport", StructType([
            StructField("name", StringType(), True),
            StructField("iata", StringType(), True),
            StructField("icao", StringType(), True),
            StructField("latitude", DoubleType(), True),
            StructField("longitude", DoubleType(), True),
            StructField("geo_region", StringType(), True)
        ]), True),
        StructField("scheduled_time", StringType(), True),
        StructField("estimated_time", StringType(), True)
    ]), True)
])

# Convert the API response JSON data to a PySpark DataFrame
df = spark.read.json(sc.parallelize([data]), schema=schema)

# Define the range of coordinates that define Poland's boundaries
min_lat = 49.002698
max_lat = 54.836579
min_lon = 14.122970
max_lon = 24.029986

# Filter the DataFrame to only include flights passing over Poland
filtered_df = df.filter((col("departure.airport.latitude") > min_lat) & (col("departure.airport.latitude") < max_lat) & (col("departure.airport.longitude") > min_lon) & (col("departure.airport.longitude") < max_lon))

# Show the filtered DataFrame
filtered_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------+---------+-------+
|flight_date|flight_status|departure|arrival|
+-----------+-------------+---------+-------+
+-----------+-------------+---------+-------+

In [13]:
import matplotlib.pyplot as plt

# Get the latitude and longitude of the departure airports
lats = filtered_df.select("departure.airport.latitude").collect()
lons = filtered_df.select("departure.airport.longitude").collect()

# Create a scatter plot of the flight paths
plt.scatter(lons, lats)
plt.title("Flights over Poland")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'matplotlib'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'matplotlib'

